## Voting Ensemble(투표 기반 앙상블)
- 여러 분류기를 학습시킨 후 각각의 분류기가 예측하는 레이블의 범주가 가장 많이 나오는 범주를 예측하는 방법 
- 방법 
    - 개별 분류기의 최적 하이퍼 파라미ㅣ터를 찾은 후 투표 기반 앙상블로 모델을 구축 
    - 좀 더 좋은 분류와 회귀 예측 수행 

### Classification

In [1]:
import warnings 
warnings.filterwarnings("ignore")
import pandas as pd 

data = pd.read_csv("../Data/breast-cancer-wisconsin.csv")
X = data.iloc[:, 1:10]
y = data[['Class']]

from sklearn.model_selection import * 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 410)

from sklearn.preprocessing import * 
minmax = MinMaxScaler()
minmax.fit(X_train)
X_scaled_train = minmax.transform(X_train)
X_scaled_test = minmax.transform(X_test)

In [5]:
import sklearn
dir(sklearn)

['__SKLEARN_SETUP__',
 '__all__',
 '__builtins__',
 '__cached__',
 '__check_build',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_distributor_init',
 '_isotonic',
 '_loss',
 'base',
 'calibration',
 'clone',
 'cluster',
 'compose',
 'config_context',
 'covariance',
 'cross_decomposition',
 'datasets',
 'decomposition',
 'discriminant_analysis',
 'dummy',
 'ensemble',
 'exceptions',
 'experimental',
 'externals',
 'feature_extraction',
 'feature_selection',
 'gaussian_process',
 'get_config',
 'impute',
 'inspection',
 'isotonic',
 'kernel_approximation',
 'kernel_ridge',
 'linear_model',
 'logger',
 'logging',
 'manifold',
 'metrics',
 'mixture',
 'model_selection',
 'multiclass',
 'multioutput',
 'naive_bayes',
 'neighbors',
 'neural_network',
 'os',
 'pipeline',
 'preprocessing',
 'random',
 'random_projection',
 'semi_supervised',
 'set_config',
 'setup_module',
 'show_versions',
 'svm',
 'sys',
 'tree',

In [7]:
from sklearn.ensemble import * 
from sklearn.linear_model import * 
from sklearn.svm import * 

randomforest_model = RandomForestClassifier(random_state=410)
logistic_model = LogisticRegression(random_state=410)
SVM_model = SVC(random_state=410)

In [9]:
voting_hard = VotingClassifier(
    estimators=[("lr", logistic_model), ("rf", randomforest_model), ("svc", SVM_model)],
    voting="hard"
)
voting_hard.fit(X_scaled_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=410)),
                             ('rf', RandomForestClassifier(random_state=410)),
                             ('svc', SVC(random_state=410))])

In [10]:
from sklearn.metrics import *
for clf in (logistic_model, randomforest_model, SVM_model, voting_hard):
    clf.fit(X_scaled_train, y_train)
    y_pred = clf.predict(X_scaled_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9824561403508771
RandomForestClassifier 0.9766081871345029
SVC 0.9707602339181286
VotingClassifier 0.9766081871345029


In [13]:
randomforest_model = RandomForestClassifier(random_state=410)
logistic_model = LogisticRegression(random_state=410)
SVM_model = SVC(probability=True, random_state=410)

In [14]:
voting_soft = VotingClassifier(
    estimators=[("lr", logistic_model), ("rf", randomforest_model), ("svc", SVM_model)],
    voting="soft"
)
voting_soft.fit(X_scaled_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=410)),
                             ('rf', RandomForestClassifier(random_state=410)),
                             ('svc', SVC(probability=True, random_state=410))],
                 voting='soft')

In [15]:
for clf in (logistic_model, randomforest_model, SVM_model, voting_soft):
    clf.fit(X_scaled_train, y_train)
    y_pred = clf.predict(X_scaled_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9824561403508771
RandomForestClassifier 0.9766081871345029
SVC 0.9707602339181286
VotingClassifier 0.9707602339181286


### Regression

In [16]:
import pandas as pd 
data2 = pd.read_csv("../Data/house_price.csv", encoding="utf-8")
X = data2.iloc[:, 1:5]
y = data2[['house_value']]

X_train, X_test ,y_train, y_test = train_test_split(X, y, random_state=410)

minmax = MinMaxScaler()
minmax.fit(X_train)
X_scaled_train = minmax.transform(X_train)
X_scaled_test =  minmax.transform(X_test)


In [21]:
from sklearn.linear_model import * 
from sklearn.ensemble import * 

LRM = LinearRegression()
RFM = RandomForestRegressor(random_state = 410)

voting_Regressor = VotingRegressor(estimators=  [("lr", LRM), ("rf", RFM)])
voting_Regressor.fit(X_scaled_train, y_train)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(random_state=410))])

In [22]:
pred_train = voting_Regressor.predict(X_scaled_train)
voting_Regressor.score(X_scaled_train, y_train)

0.7976653151568789

In [23]:
pred_test = voting_Regressor.predict(X_scaled_test)
voting_Regressor.score(X_scaled_test, y_test)

0.5914394987387273

In [25]:
import numpy as np 
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)

print("학습 데이터 RMSE : ", np.sqrt(MSE_train))
print("테스트 데이터 RMSE : ", np.sqrt(MSE_test))

학습 데이터 RMSE :  42674.66323816914
테스트 데이터 RMSE :  62158.44852522241
